In [1]:
# for reading and validating data
import emeval.input.spec_details as eisd
import emeval.input.phone_view as eipv
import emeval.input.eval_view as eiev

In [2]:
# Visualization helpers
import emeval.viz.phone_view as ezpv
import emeval.viz.eval_view as ezev

In [3]:
# for pipelined data
import emeval.analysed.phone_view as eapv

In [4]:
import pandas as pd

In [5]:
import geopandas as gpd
import shapely as shp

In [6]:
import numpy as np

In [7]:
import emeval.metrics.dist_calculations as emd

In [8]:
# For plots
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
# Analytics results
import emeval.metrics.segmentation as ems

In [10]:
# For easier debugging while working on modules
import importlib

In [11]:
def import_sd_and_pv_from_server(trips  = ["unimodal_trip_car_bike_mtv_la", "car_scooter_brex_san_jose", "train_bus_ebike_mtv_ucb"], 
                                 AUTHOR_EMAIL  = "shankari@eecs.berkeley.edu", 
                                 DATASTORE_LOC = "http://localhost:8080", 
                                 pkl_file_name = None):
    sd_l = []
    pv_l = []
    for trip in trips:
        sd = eisd.ServerSpecDetails(DATASTORE_LOC, AUTHOR_EMAIL, trip)
        pv = eipv.PhoneView(sd)
        sd_l.append(sd)
        pv_l.append(pv)
    if pkl_file_name:
        import pickle
        with open(pkl_file_name, 'wb') as outp:
            for pv in pv_l:
                pickle.dump(pv, outp, pickle.HIGHEST_PROTOCOL)
    return sd_l, pv_l

In [12]:
def import_pv_from_pkl(pkl_file_name, 
                       trips = ["unimodal_trip_car_bike_mtv_la", "car_scooter_brex_san_jose", "train_bus_ebike_mtv_ucb"]):
    import pickle
    pv_l = []
    with open('pv.pkl', 'rb') as inp:
        for trip in trips:
            pv_l.append(pickle.load(inp))
    return pv_l

In [13]:
(pv_la, pv_sj, pv_ucb) = import_pv_from_pkl('pv.pkl')

In [16]:
import emeval.analysed.location_smoothing as eaicl

In [18]:
eaicl.add_dist_heading_speed()

AttributeError: 'str' object has no attribute 'to_dict'

In [20]:
for phone_os, phone_map in pv.map().items():
    for phone_label, phone_detail_map in phone_map.items():
        for (r_idx, r) in enumerate(phone_detail_map["evaluation_ranges"]):
            run_errors = []
            for (tr_idx, tr) in enumerate(r["evaluation_trip_ranges"]):
                trip_errors = []
                for (sr_idx, sr) in enumerate(tr["evaluation_section_ranges"]):
                    pass

NameError: name 'pv' is not defined